In [1]:
import openai

openai.api_key = 'sk-XXX'

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

### First solution for stage 1 of usecase:
- Switching between retrieval chains as intent recognition
- RetrievalQA chain uses load_qa_chain under the hood, retrieving relevant chunks of text and feeding those to the language model

In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

In [4]:
# Now we can load the persisted database from disk, and use it as normal. 
vectorstore = Chroma(persist_directory='./chroma_storage/', embedding_function=embeddings)
vectorstore_intents = Chroma(persist_directory='./chroma_storage_intents/', embedding_function=embeddings)

In [5]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True,
    input_key='question', output_key='answer'
)

In [6]:
qa_raipay = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0, openai_api_key=openai.api_key), 
    vectorstore.as_retriever(), 
    memory=memory,
    return_source_documents=True
)

In [7]:
query = "Kannen sie im ausland RAIPAY nutze?"
result = qa_raipay({"question": query})
result

{'question': 'Kannen sie im ausland RAIPAY nutze?',
 'chat_history': [HumanMessage(content='Kannen sie im ausland RAIPAY nutze?', additional_kwargs={}, example=False),
  AIMessage(content=' Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.', additional_kwargs={}, example=False)],
 'answer': ' Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.',
 'source_documents': [Document(page_content='KANN ICH MIT RAIPAY AUCH IM AUSLAND ZAHLEN? Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.', metada

In [8]:
qa_open_block = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0, openai_api_key=openai.api_key), 
    vectorstore_intents.as_retriever(), 
    memory=memory,
    return_source_documents=True
)

In [9]:
query = "Kannen ich einen account geofnen?"
result = qa_open_block({"question": query})
result

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'question': 'Kannen ich einen account geofnen?',
 'chat_history': [HumanMessage(content='Kannen sie im ausland RAIPAY nutze?', additional_kwargs={}, example=False),
  AIMessage(content=' Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.', additional_kwargs={}, example=False),
  HumanMessage(content='Kannen ich einen account geofnen?', additional_kwargs={}, example=False),
  AIMessage(content=' Nein, ich kenne keine Möglichkeit, ein RaiPay-Konto zu eröffnen.', additional_kwargs={}, example=False)],
 'answer': ' Nein, ich kenne keine Möglichkeit, ein RaiPay-Konto zu eröffnen.',
 'source_documents': [Document(page_content='Open Account In order to open an account, the operator needs to obtain the type of desired account (current or savings), the name of the client, the address of the client (street, city, postcode, countr

### Intent recognition through QA selector

In [10]:
from langchain import PromptTemplate
from langchain.chains import ConversationChain

In [29]:
retriever_chains = [
    {
        "name": "RAIPAY mobile banking application", 
        "description": "Good for answering questions about mobile banking and RAIPAY mobile banking application", 
        "retriever": qa_raipay
    },
    {
        "name": "Open or block account", 
        "description": "Good for finding out what to do and what to ask if one wants to open a new bank account or block his bank card", 
        "retriever": qa_open_block
    }
]

DEFAULT_TEMPLATE = """You are an intent recognizing AI.

Based on human input choose only one of these intents:
open bank account
block bank card
asking questions about RAIPAY banking application
chitchat

Current conversation:
{chat_history}
Human: {question}
AI:"""

memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True,
    input_key='question', output_key='answer'
)

prompt_default_template = DEFAULT_TEMPLATE.replace('input', 'question')

prompt_default = PromptTemplate(
    template=prompt_default_template, input_variables=['chat_history', 'question']
)
default_chain=ConversationChain(
    llm=OpenAI(openai_api_key=openai.api_key), 
    prompt=prompt_default, 
    input_key='question', 
    output_key='answer',
    memory=memory
)


In [31]:
print(default_chain.run('Wie gehts?'))

 Chitchat.
